In [2]:
pip install psycopg2-binary


     ---------------------------------------- 1.2/1.2 MB 8.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
import psycopg2
import pandas as pd
import requests
from io import StringIO

In [ ]:
query = '''
 WITH t AS (
  SELECT 
    price, 
    row_number() OVER (ORDER BY price) AS number 
  FROM (
    SELECT 
      order_id, 
      product_ids, 
      SUM(price) AS price 
    FROM (
      SELECT 
        *, 
        unnest(product_ids) AS product_id 
      FROM 
        orders
    ) t1 
    LEFT JOIN products USING(product_id) 
    WHERE 
      order_id NOT IN (
        SELECT 
          order_id 
        FROM 
          user_actions 
        WHERE 
          action = 'cancel_order'
      ) 
    GROUP BY 
      order_id, 
      product_ids
  ) AS t2
) 
SELECT 
  CASE 
    WHEN (SELECT COUNT(*) FROM t) % 2 = 0 THEN (
      SELECT 
        AVG(price) 
      FROM 
        t 
      WHERE 
        number = (SELECT MAX(number)/2.0 FROM t) 
        OR number = (SELECT MAX(number)/2.0 FROM t) + 1
    ) 
    ELSE (
      SELECT 
        price 
      FROM 
        t 
      WHERE 
        number = (SELECT AVG(number) FROM t)
    ) 
  END AS median_price 
FROM 
  t 
LIMIT 1;

    '''

In [14]:
# URL для выполнения запроса
redash_url = 'http://redash.public.karpov.courses/api/queries/51119/results.csv?api_key=nN4z8WpasuiJGBJnWfXGJl4r4BTuAfRh0hb0eYPZ'

# Выполнение запроса и получение данных в формате CSV
response = requests.get(redash_url)

# Проверка успешности выполнения запроса
if response.status_code == 200:
    # Преобразование данных в DataFrame
    df = pd.read_csv(StringIO(response.text))
    
    # Вывод первых 5 строк DataFrame
    print(df.head())
else:
    print(f"Error: {response.status_code}")
    print(response.text)

   median_price
0         321.0


**Вывод:** c помощью оконной функции рассчитал медианную стоимость всех заказов из таблицы orders, оформленных в сервисе. Запрос учитывает два возможных сценария: для чётного и нечётного числа заказов без применения встроенных функции для расчёта квантилей.